In [2]:
import numpy as np
from dx import *

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.5.0


In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.36159882, -0.42213982, -0.69078064,  0.44089812,  0.16006881,
        -0.30860293, -0.25806767, -0.35691074,  0.08809576,  0.5278244 ]],
      dtype=float32)

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.07274594, 0.0684725 , 0.05234159, 0.16230382, 0.12256491,
        0.07670517, 0.08068109, 0.07308778, 0.11405351, 0.17704365]],
      dtype=float32)

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.5677862

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2907 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1419 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1048 - accuracy: 0.9681
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0869 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0718 - accuracy: 0.9773: 0s - los


In [12]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0689 - accuracy: 0.9789


[0.06890139728784561, 0.9789000153541565]

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.42349793e-08, 1.89427984e-09, 3.67462417e-05, 7.52552878e-05,
        1.11072540e-10, 2.14351280e-06, 1.68366663e-13, 9.99885321e-01,
        2.56602277e-07, 2.53002781e-07],
       [4.57646962e-07, 1.14239745e-04, 9.99876142e-01, 7.10192035e-06,
        5.34541648e-18, 2.02025763e-06, 6.88782631e-09, 6.38554010e-12,
        5.52551924e-08, 1.19008112e-14],
       [5.67823918e-06, 9.95863438e-01, 1.78802788e-04, 2.93183675e-05,
        1.02508340e-04, 3.63465733e-05, 2.23462666e-05, 3.56509443e-03,
        1.92190870e-04, 4.24969539e-06],
       [9.99956608e-01, 3.62421232e-10, 2.94543170e-05, 2.59671395e-08,
        1.55772444e-08, 9.49729326e-07, 3.71783335e-06, 7.27090901e-06,
        3.77801648e-08, 2.01737043e-06],
       [6.71392115e-07, 4.87749674e-10, 2.09616246e-07, 2.08610427e-08,
        9.95723844e-01, 3.20036264e-07, 1.31472507e-06, 1.06856845e-04,
        1.58421653e-06, 4.16510506e-03]], dtype=float32)>

## Advanced version 

In [14]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [15]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [16]:
x_train.shape

(60000, 28, 28, 1)

In [17]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [18]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [19]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [20]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [21]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [22]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )